## 4.1 将LLM接入LangChain

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv('.env.local'))

api_key = os.getenv("DEEPSEEK_API_KEY")
api_url = os.getenv("DEEPSEEK_API_URL")
model = os.getenv("DEEPSEEK_MODEL")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=api_key,
    base_url=api_url,
    model_name=model,
    temperature=0.0,
)

/Users/young/project/llmProject/llm-universe/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
output = llm.invoke("请你自我介绍一下自己！")

In [5]:
output

AIMessage(content='你好呀！我是 **DeepSeek Chat**，由深度求索（DeepSeek）公司打造的智能 AI 助手。我的最新版本是 **DeepSeek-V3**，知识截止到 **2024年7月**，拥有 **128K 上下文记忆**，可以处理超长文本，还能阅读和解析 **PDF、Word、Excel、PPT、TXT** 等文件内容。  \n\n### **我的特点：**  \n🔹 **免费使用**：目前不收费，随时为你解答问题！  \n🔹 **超长上下文**：支持长达 128K 的对话记忆，适合处理复杂任务。  \n🔹 **文件阅读**：可以帮你分析文档内容，提取关键信息。  \n🔹 **知识丰富**：覆盖科技、编程、学习、生活、娱乐等多个领域。  \n🔹 **逻辑清晰**：擅长推理、写作、翻译、代码编写等任务。  \n\n无论是学习、工作，还是日常生活中的疑问，都可以来找我聊聊！😊 你今天有什么想了解的呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 8, 'total_tokens': 226, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'finish_reason': 'stop', 'logprobs': None}, id='run-1553582b-f98b-473b-9a15-1a507ab9ad7b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 218, 'total_tokens': 226})

In [8]:
# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

In [9]:
# 使用prompt模板
from langchain_core.prompts import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate([
    ("system", template),
    ("human", human_template)
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"

messages = chat_prompt.invoke({"input_language": "中文", "output_language": "英文", "text": text})
messages

ChatPromptValue(messages=[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.', additional_kwargs={}, response_metadata={}), HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。', additional_kwargs={}, response_metadata={})])

In [10]:
# 测试一下
output = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.  \n\n(Note: This translation aims to preserve the poetic and somewhat surreal imagery of the original while ensuring clarity in English. The phrase "比身体重的行李" is rendered as "luggage heavier than my body" to convey both literal and metaphorical weight. "游入尼罗河底" becomes "Dived into the depths of the Nile" to maintain the sense of submersion and mystery. "光圈" is translated as "halos" to evoke a luminous, almost otherworldly visual, fitting the dreamlike tone.)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 45, 'total_tokens': 195, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 45}, 'model_name': 'deepseek-chat'

In [11]:
# 格式化输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.  \n\n(Note: This translation aims to preserve the poetic and somewhat surreal imagery of the original while ensuring clarity in English. The phrase "比身体重的行李" is rendered as "luggage heavier than my body" to convey both literal and metaphorical weight. "游入尼罗河底" becomes "Dived into the depths of the Nile" to maintain the sense of submersion and mystery. "光圈" is translated as "halos" to evoke a luminous, almost otherworldly visual, fitting the dreamlike tone.)'

In [12]:
# 使用LCEL表达式
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language": "中文", "output_language": "英文", "text": text})

'I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.'

In [13]:
# 再试一下，英译中
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language": "英文", "output_language": "中文","text": text})

'我扛着比身体还重的行李潜入尼罗河底，穿过几道闪电后看见一堆光晕，不知是不是这里。'

## 4.2 构建检索问答链

### 4.2.1 加载向量数据库

In [15]:
from ark_embedding import ArkEmbeddings
from langchain.vectorstores.chroma import Chroma

import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv('.env.local'))

embedding_api_key = os.getenv("ARK_API_KEY")
embedding_api_url = os.getenv("ARK_API_URL")
embedding_model = os.getenv("ARK_EMBEDDING_MODEL")

In [16]:
# 初始化 Embeddings
embedding = ArkEmbeddings(
    api_key=embedding_api_key,
    api_url=embedding_api_url,
    model=embedding_model,
)

# 向量数据库持久化路径
persist_directory = "../data_base/vector_db/chroma"

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
)

/var/folders/7r/h44tfn_x35jg4s7msmb1gdzw0000gp/T/ipykernel_7361/801487602.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(


In [17]:
print(f"向量库中存储的数据：{vectordb._collection.count()}")

向量库中存储的数据：1004


In [18]:
# 检索相似文档
question = "什么是prompt engineering?"
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
docs = retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


In [19]:
# 打印检索内容
for i, doc in enumerate(docs):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第1个内容: 
 具体来说，首先编写初版 Prompt，然后通过多轮调整逐步改进，直到生成了满意的结果。对于更复杂的应用，可以在多个样本上进行迭代训练，评估 Prompt 的平均表现。在应用较为成熟后，才需要采用在多个样本集上评估 Prompt 性能的方式来进行细致优化。因为这需要较高的计算资源。

总之，Prompt 工程师的核心是掌握 Prompt 的迭代开发和优化技巧，而非一开始就要求100%完美。通过不断调整试错，最终找到可靠适用的 Prompt 形式才是设计 Prompt 的正确方法。

读者可以在 Jupyter Notebook 上，对本章给出的示例进行实践，修改 Prompt 并观察不同输出，以深入理解 Prompt 迭代优化的过程。这会对进一步开发复杂语言模型应用提供很好的实践准备。

三、英文版

产品说明书
-----------------------------------------------------
检索到的第2个内容: 
 第一章 简介

欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Large Language Model， 大语言模型）技术在产品中的应用方面做出了很大贡献。她还参与编写了教授人们使用 Prompt 的 OpenAI cookbook。我们希望通过本模块的学习，与大家分享使用提示词开发 LLM 应用的最佳实践和技巧。
-----------------------------------------------------
检索到的第3个内容: 
 第二章 提示原则

如何去使用 Prompt，以充分发挥 LLM 的性能？首先我们需要知道设计 Prompt 的原则，它们是每一个开发者设计 Prompt 所必须知道的基础概念。本章讨论了设计高效 Prompt 的两个关键原则：编写清晰、具体的指令和给予模型充足思考时间。掌握这两点，对创建可靠的语言模型交互尤为重要。

首先，Prom

### 4.2.2 创建检索链

In [20]:
from langchain_core.runnables import RunnableLambda

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner

retrieval_chain.invoke("南瓜书是什么？")

'5.1 综合样例\n\n最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases\n编委会\n主编：Sm1les、archwalker、jbb0523\n编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226\n封面设计：构思-Sm1les、创作-林王茂盛\n致谢\n特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、\nspareribs、sunchaothu、StevenLzq 在最早期的时候对南瓜书所做的贡献。\n扫描下方二维码，然后回复关键词“南瓜书”，即可加入“南瓜书读者交流群”\n版权声明\n本作品采用知识共享署名-非商业性使用-相同方式共享4.0 国际许可协议进行许可。\n\n←_←'